# Exercise 8

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import disk, dilation
from skimage.measure import label
from skimage.color import label2rgb
from skimage.filters import threshold_otsu
from scipy.ndimage import convolve

FS = 15 # Fontsize of caption

## 8.1 Grayscale edge detector

In [ ]:
# Load the image in grayscale
I = cv2.imread('bike.png', cv2.IMREAD_GRAYSCALE)

# Normalize the image to 0-1
I = I.astype(np.float32) / 255.0

# Define the structuring element (rectangle with size 4x5)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 4))

# Perform dilation on the original image
I_dilated = cv2.dilate(I, kernel)

# Perform edge detection by subtracting the original image from the dilated image
edge = I_dilated - I

# Threshold to convert to binary image
BW = edge > 0.2  # 0.2 is the threshold value

# Show images
plt.figure(figsize = (10, 10))

plt.subplot(2, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original Image', fontsize = FS)

plt.subplot(2, 2, 2)
plt.imshow(I_dilated, cmap = 'gray')
plt.title('Dilated Image', fontsize = FS)

plt.subplot(2, 2, 3)
plt.imshow(edge, cmap = 'gray')
plt.title('Edge = Dilation - Origin', fontsize = FS)

plt.subplot(2, 2, 4)
plt.imshow(BW, cmap = 'gray')
plt.title('BW of Edge', fontsize = FS)

plt.tight_layout()
plt.show()


## 8.2 Coins couting using dilation

In [ ]:
# Load the test image
I = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)

# Binarize the input image
level = threshold_otsu(I)
BW = 1 - (I > level).astype(np.uint8)  # Revert black and white

# a. No dilation
L = label(BW, connectivity=2)  # Connectivity=2 for 8-connected neighborhood
obj = len(np.unique(L)) - 1
rgbLabel1 = label2rgb(L, image_alpha=0)

print(f"Number of coins before dilation: {obj}")

# b. Use dilation
se = disk(25)
BW_dilated = 1 - dilation((I > level).astype(np.uint8), se)

L2 = label(BW_dilated, connectivity=2)
obj2 = len(np.unique(L2)) - 1
rgbLabel2 = label2rgb(L2, image_alpha=0)

print(f"Number of coins after dilation: {obj2}")

# Show images
plt.figure(1, figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(I, cmap='gray')
plt.title('Original image', fontsize=FS)

plt.subplot(2, 2, 2)
plt.imshow(BW, cmap='gray')
plt.title('Binary image', fontsize=FS)

plt.subplot(2, 2, 3)
plt.imshow(rgbLabel1)
plt.title('Labeling before dilation', fontsize=FS)

plt.subplot(2, 2, 4)
plt.imshow(rgbLabel2)
plt.title('Labeling after dilation', fontsize=FS)

plt.tight_layout()
plt.show()


## 8.3 

In [ ]:
# Load the test image
I = cv2.imread('butterfly.png', cv2.IMREAD_GRAYSCALE)

# Define 3 different structuring elements
se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 1))  # Line, 10 pixels, horizontal
se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))  # Line, diagonal approximation
se3 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 10))  # Line, 10 pixels, vertical

# Perform a sequence of dilation
I_dilation1 = cv2.dilate(I, se1)
I_dilation2 = cv2.dilate(I_dilation1, se2)
I_dilation3 = cv2.dilate(I_dilation2, se3)

# Show images
plt.figure(1, figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(I, cmap='gray')
plt.title('Original image', fontsize=FS)

plt.subplot(2, 2, 2)
plt.imshow(I_dilation1, cmap='gray')
plt.title('1st dilation image', fontsize=FS)

plt.subplot(2, 2, 3)
plt.imshow(I_dilation2, cmap='gray')
plt.title('2nd dilation image', fontsize=FS)

plt.subplot(2, 2, 4)
plt.imshow(I_dilation3, cmap='gray')
plt.title('3rd dilation image', fontsize=FS)

plt.tight_layout()
plt.show()


## 8.4 Change object's feature of color image

In [ ]:
# Load the test image
I = cv2.imread('shapes.png')
I_rgb = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display

# Convert to grayscale image
I_grayscale = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

# Binarize the grayscale image
level = threshold_otsu(I_grayscale)
I_BW = 1 - (I_grayscale > level).astype(np.uint8)

# Label the objects
L = label(I_BW, connectivity=2)  # 8-connectivity
obj = np.max(L)

# Define the color map
cmap = np.array([
    [255, 0, 0],    # Circle -> Red
    [255, 0, 255],  # Square -> Magenta
    [255, 255, 0],  # Rectangle -> Yellow
    [0, 255, 255],  # Asterisk -> Cyan
    [0, 0, 0]       # Pentagon -> Black
]) / 255.0  # Normalize for matplotlib

# Apply the color map to labeled regions
rgbLabel = label2rgb(L, colors=cmap[:obj], bg_label=0)

# Show images
plt.figure(1, figsize=(12, 12))
plt.subplot(2, 2, 1)
plt.imshow(I_rgb)
plt.title('Original image', fontsize=FS)

plt.subplot(2, 2, 2)
plt.imshow(I_grayscale, cmap='gray')
plt.title('Grayscale image', fontsize=FS)

plt.subplot(2, 2, 3)
plt.imshow(I_BW, cmap='gray')
plt.title('Binary image', fontsize=FS)

plt.subplot(2, 2, 4)
plt.imshow(rgbLabel)
plt.title('Converted color image', fontsize=FS)

plt.tight_layout()
plt.show()


## 8.5 Edge detection: Prewitt operator

In [ ]:
# Load the test image
O_I = cv2.imread('bike.png', cv2.IMREAD_GRAYSCALE)
I = O_I.astype(np.float64)  # Convert to double for consistency

# Define Prewitt filter
h = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])  # Horizontal Prewitt

# Apply Prewitt filter in horizontal direction
filtered_I1 = convolve(I, h, mode='mirror')
filtered_I1 = np.abs(filtered_I1)  # Get its magnitude
filtered_I1 = filtered_I1 / np.max(filtered_I1)  # Normalize

# Apply Prewitt filter in vertical direction
filtered_I2 = convolve(I, h.T, mode='mirror')  # Transpose for vertical filter
filtered_I2 = np.abs(filtered_I2)  # Get its magnitude
filtered_I2 = filtered_I2 / np.max(filtered_I2)  # Normalize

# Combine edges in both directions
edge_sum = filtered_I1 ** 2 + filtered_I2 ** 2
edge_sum = np.log(edge_sum + 1)  # Magnitude in log
log_edge_sum = edge_sum / np.max(edge_sum)  # Normalize

# Thresholding
bw_edge1 = log_edge_sum > 0.01  # Threshold = 1%
bw_edge2 = log_edge_sum > 0.05  # Threshold = 5%
bw_edge3 = log_edge_sum > 0.1   # Threshold = 10%

# Show images
plt.figure(1, figsize=(12, 8))
plt.subplot(2, 3, 1)
plt.imshow(O_I, cmap='gray')
plt.title('Original image', fontsize=FS)

plt.subplot(2, 3, 2)
plt.imshow(filtered_I1, cmap='gray')
plt.title('Prewitt horizontal image', fontsize=FS)

plt.subplot(2, 3, 3)
plt.imshow(filtered_I2, cmap='gray')
plt.title('Prewitt vertical image', fontsize=FS)

plt.subplot(2, 3, 4)
plt.imshow(bw_edge1, cmap='gray')
plt.title('Mag > 0.01 image', fontsize=FS)

plt.subplot(2, 3, 5)
plt.imshow(bw_edge2, cmap='gray')
plt.title('Mag > 0.05 image', fontsize=FS)

plt.subplot(2, 3, 6)
plt.imshow(bw_edge3, cmap='gray')
plt.title('Mag > 0.1 image', fontsize=FS)

plt.tight_layout()
plt.show()


## 8.6 Edge detection: Sobel operator

In [ ]:
# Load the test image
O_I = cv2.imread('man_face.png', cv2.IMREAD_GRAYSCALE)
I = O_I.astype(np.float64)  # Convert to double for consistency

# Define Sobel filters
h = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])  # Horizontal Sobel filter

# Apply Sobel filter in horizontal direction
filtered_I1 = convolve(I, h, mode='mirror')  # Symmetric padding
filtered_I1 = np.abs(filtered_I1)  # Get its magnitude
filtered_I1 = filtered_I1 / np.max(filtered_I1)  # Normalize

# Apply Sobel filter in vertical direction
filtered_I2 = convolve(I, h.T, mode='mirror')  # Transpose for vertical filter
filtered_I2 = np.abs(filtered_I2)  # Get its magnitude
filtered_I2 = filtered_I2 / np.max(filtered_I2)  # Normalize

# Combine edges in both directions
edge_sum = filtered_I1 ** 2 + filtered_I2 ** 2
edge_sum = np.log(edge_sum + 1)  # Magnitude in log
log_edge_sum = edge_sum / np.max(edge_sum)  # Normalize

# Thresholding
bw_edge1 = log_edge_sum > 0.01  # Threshold = 1%
bw_edge2 = log_edge_sum > 0.05  # Threshold = 5%
bw_edge3 = log_edge_sum > 0.1   # Threshold = 10%

# Show images
plt.figure(1, figsize=(12, 8))
plt.subplot(2, 3, 1)
plt.imshow(O_I, cmap='gray')
plt.title('Original image', fontsize=FS)

plt.subplot(2, 3, 2)
plt.imshow(filtered_I1, cmap='gray')
plt.title('Sobel horizontal image', fontsize=FS)

plt.subplot(2, 3, 3)
plt.imshow(filtered_I2, cmap='gray')
plt.title('Sobel vertical image', fontsize=FS)

plt.subplot(2, 3, 4)
plt.imshow(bw_edge1, cmap='gray')
plt.title('Mag > 0.01 image', fontsize=FS)

plt.subplot(2, 3, 5)
plt.imshow(bw_edge2, cmap='gray')
plt.title('Mag > 0.05 image', fontsize=FS)

plt.subplot(2, 3, 6)
plt.imshow(bw_edge3, cmap='gray')
plt.title('Mag > 0.1 image', fontsize=FS)

plt.tight_layout()
plt.show()
